# Chatbot with Memory
Chatbots are everywhere, but unlike a lot of people think, LLMs like the model behind ChatGPT do not remember anything you say to them. The reason these apps can remember what you said previously is because the apps themselves store information about you or send the chat history to the model every time. With the code below you can create a very simple chatbot on tops of the orchestration service. But something is wrong with the memory of the chatbot. Can you fix it?

In [30]:
import os
import json
from typing import List

ROOT_PATH_DIR = os.path.dirname(os.getcwd())
AICORE_CONFIG_FILENAME = 'service-key.json'
with open(os.path.join(ROOT_PATH_DIR, AICORE_CONFIG_FILENAME), 'r') as config_file:
    config_data = json.load(config_file)

os.environ["AICORE_AUTH_URL"]=config_data["url"]+"/oauth/token"
os.environ["AICORE_CLIENT_ID"]=config_data["clientid"]
os.environ["AICORE_CLIENT_SECRET"]=config_data["clientsecret"]
os.environ["AICORE_BASE_URL"]=config_data["serviceurls"]["AI_API_URL"]
os.environ["AICORE_RESOURCE_GROUP"]="default"
AICORE_ORCHESTRATION_DEPLOYMENT_URL = ""

In [31]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

In [32]:
class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gpt-35-turbo"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        # An LLM does not remember what you say/ask unless you feed the conversation history to the model each time. Something is off here though :S Can you help?
        print(self.history)
        self.history = response.module_results.templating
        history.append(message)
        print(self.history)

        return message.content
    
    def reset(self):
        self.history = []

service = OrchestrationService(api_url=AICORE_ORCHESTRATION_DEPLOYMENT_URL)
bot = ChatBot(orchestration_service=service)

In [ ]:
bot.chat("Hello, how are you?")

In [ ]:
bot.chat("What's the weather like today?")

In [ ]:
bot.chat("Can you remember what I first asked you?")

In [ ]:
bot.history